In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
from shapely.geometry import Point, LineString
import difflib

## Lyon

In [2]:
accidents_lyon=gpd.read_file('data_lyon.gpkg')
accidents_lyon=accidents_lyon[['Num_Acc_x','int','geometry','adr']].drop_duplicates('Num_Acc_x')

In [3]:
chaussees=gpd.read_file('lyon_network.gpkg')

In [4]:
accidents_lyon=accidents_lyon.to_crs(chaussees.crs)
distances = accidents_lyon.geometry.apply(lambda point: chaussees.distance(point))
dist_transpose=np.transpose(distances)

In [5]:
nearest_indices = dist_transpose.apply(lambda dist: dist.idxmin())

# Assign attributes of nearest features to accident points
nearest_features = chaussees.iloc[nearest_indices]  # Flatten indices to 1D array for indexing
nearest_features_reset  = nearest_features.reset_index(drop=True)

nearest_features = nearest_features.drop(columns='geometry')

accidents_lyon = pd.concat([accidents_lyon.reset_index(drop=True), nearest_features.reset_index(drop=True)], axis=1)


In [7]:
streets=gpd.read_file('streets_lyon.geojson')
streets=streets[['highway','geometry']]
streets.to_crs(chaussees.crs, inplace=True)

In [8]:
distances_lyonstreet = accidents_lyon.geometry.apply(lambda point: streets.distance(point))
dist_transpose=np.transpose(distances_lyonstreet)


In [9]:
dist_transpose=np.transpose(distances_lyonstreet)


In [10]:
nearest_indices = dist_transpose.apply(lambda dist: dist.idxmin())

# Assign attributes of nearest features to accident points
nearest_features = streets.iloc[nearest_indices]  # Flatten indices to 1D array for indexing
nearest_features_reset  = nearest_features.reset_index(drop=True)

nearest_features = nearest_features.drop(columns='geometry')

accidents_lyon = pd.concat([accidents_lyon.reset_index(drop=True), nearest_features.reset_index(drop=True)], axis=1)

## Paris

In [11]:
accidents_paris=gpd.read_file('data_paris.gpkg')
accidents_paris=accidents_paris[['Num_Acc_x','int','geometry']].drop_duplicates('Num_Acc_x')


In [12]:
reseau_cyclable=gpd.read_file('amenagements-velo-en-ile-de-france.geojson').to_crs('EPSG:2154')

In [13]:
paris=gpd.read_file('paris.geojson').to_crs('EPSG:2154')

Skipping field reg_code: unsupported OGR type: 5
Skipping field reg_name: unsupported OGR type: 5
Skipping field dep_code: unsupported OGR type: 5
Skipping field dep_name: unsupported OGR type: 5
Skipping field epci_code: unsupported OGR type: 5
Skipping field epci_curren: unsupported OGR type: 5
Skipping field epci_name: unsupported OGR type: 5


In [14]:
reseau_cyclable_paris=gpd.sjoin(reseau_cyclable,paris[['geometry','epci_name_u']])

In [15]:
accidents_paris.to_crs('EPSG:3035',inplace=True)
reseau_cyclable_paris.to_crs('EPSG:3035',inplace=True)

In [16]:
distances_paris = accidents_paris.geometry.apply(lambda point: reseau_cyclable_paris.distance(point))


/opt/anaconda3/envs/tfe/lib/python3.13/site-packages/shapely/measurement.py:72: RuntimeWarning: invalid value encountered in distance
  return lib.distance(a, b, **kwargs)


In [17]:
# Transpose the distances DataFrame
dist_transpose_paris = np.transpose(distances_paris)

# Find the index of the nearest cycling path for each accident point
nearest_indices = dist_transpose_paris.apply(lambda dist: dist.idxmin())


In [18]:
# Extract the attributes of the nearest cycling paths
nearest_features = reseau_cyclable.iloc[nearest_indices]


In [19]:
# Reset the index of nearest_features to align with the indices of accident_sur_piste
nearest_features_reset = nearest_features.reset_index(drop=True)

nearest_features = nearest_features.drop(columns='geometry')


In [20]:
# Assign the nearest cycling path ID to each accident point
accidents_paris = pd.concat([accidents_paris.reset_index(drop=True), nearest_features.reset_index(drop=True)], axis=1)

In [21]:
reseau_cyclable_paris.to_crs('EPSG:4326').to_file('reseau_cyclable_paris.geojson')

# Intersections

In [22]:
accidents_lyon=accidents_lyon.to_crs('EPSG:3035')

In [23]:
acc_lyon_int=accidents_lyon.loc[(accidents_lyon['int']==2) | (accidents_lyon['int']== 3) | (accidents_lyon['int']==4) | (accidents_lyon['int']==5) ].drop_duplicates('Num_Acc_x')

In [24]:
acc_lyon_buffered = acc_lyon_int.to_crs('EPSG:3035').buffer(25)

acc_lyon_buffered_gdf = gpd.GeoDataFrame(acc_lyon_int,geometry=acc_lyon_buffered, crs='EPSG:3035')

In [25]:
lyon_traffic_signals=gpd.read_file('traffic_signals.geojson',crs='EPSG:3035')
lyon_give_way=gpd.read_file('lyon_give_way.gpkg',crs='EPSG:3035')
lyon_stops=gpd.read_file('stops.geojson',crs='EPSG:3035')

/opt/anaconda3/envs/tfe/lib/python3.13/site-packages/pyogrio/raw.py:198: RuntimeWarning: driver GeoJSON does not support open option CRS
  return ogr_read(
/opt/anaconda3/envs/tfe/lib/python3.13/site-packages/pyogrio/raw.py:198: RuntimeWarning: driver GPKG does not support open option CRS
  return ogr_read(
/opt/anaconda3/envs/tfe/lib/python3.13/site-packages/pyogrio/raw.py:198: RuntimeWarning: driver GeoJSON does not support open option CRS
  return ogr_read(


In [26]:
acc_paris_int=accidents_paris.loc[(accidents_paris['int']==2) | (accidents_paris['int']== 3) | (accidents_paris['int']==4) | (accidents_paris['int']==5)].drop_duplicates('Num_Acc_x')

In [27]:
paris_stops=gpd.read_file('paris_stops.gpkg').to_crs('EPSG:3035')
paris_give_way=gpd.read_file('paris_give_way.gpkg').to_crs('EPSG:3035')
paris_traffic_lights=gpd.read_file('paris_traffic_signals.gpkg').to_crs('EPSG:3035')

In [28]:
acc_buffered = acc_paris_int.to_crs('EPSG:3035').buffer(25)

acc_buffered_gdf = gpd.GeoDataFrame(acc_paris_int,geometry=acc_buffered, crs='EPSG:3035')

In [29]:
# Perform spatial join
acc_with_stops = gpd.sjoin(acc_lyon_buffered_gdf.drop_duplicates('Num_Acc_x'), lyon_stops.to_crs('EPSG:3035'))
# Create 'signalized' column and fill with 'yes' or 'no' based on the result of the spatial join
# Créer une colonne 'signalized' dans la GeoDataFrame intersections et assigner 'no' par défaut
acc_lyon_int['stops'] = 'no'

# Identifier les intersections avec des signaux et mettre à jour la colonne 'signalized'
acc_lyon_int.loc[acc_lyon_int['Num_Acc_x'].isin(acc_with_stops['Num_Acc_x']), 'stops'] = 'yes'

In [30]:
# Perform spatial join
acc_with_stops = gpd.sjoin(acc_buffered_gdf.drop_duplicates('Num_Acc_x'), paris_stops.to_crs('EPSG:3035'))
# Create 'signalized' column and fill with 'yes' or 'no' based on the result of the spatial join
# Créer une colonne 'signalized' dans la GeoDataFrame intersections et assigner 'no' par défaut
acc_paris_int['stops'] = 'no'

# Identifier les intersections avec des signaux et mettre à jour la colonne 'signalized'
acc_paris_int.loc[acc_paris_int['Num_Acc_x'].isin(acc_with_stops['Num_Acc_x']), 'stops'] = 'yes'

In [31]:
# Perform spatial join
acc_with_giveway = gpd.sjoin(acc_lyon_buffered_gdf.drop_duplicates('Num_Acc_x'), lyon_give_way.to_crs('EPSG:3035'))
# Create 'signalized' column and fill with 'yes' or 'no' based on the result of the spatial join
# Créer une colonne 'signalized' dans la GeoDataFrame intersections et assigner 'no' par défaut
acc_lyon_int['give_way'] = 'no'

# Identifier les intersections avec des signaux et mettre à jour la colonne 'signalized'
acc_lyon_int.loc[acc_lyon_int['Num_Acc_x'].isin(acc_with_giveway['Num_Acc_x']), 'give_way'] = 'yes'

In [32]:
# Perform spatial join
acc_with_giveway = gpd.sjoin(acc_buffered_gdf.drop_duplicates('Num_Acc_x'), paris_give_way.to_crs('EPSG:3035'))
# Create 'signalized' column and fill with 'yes' or 'no' based on the result of the spatial join
# Créer une colonne 'signalized' dans la GeoDataFrame intersections et assigner 'no' par défaut
acc_paris_int['give_way'] = 'no'

# Identifier les intersections avec des signaux et mettre à jour la colonne 'signalized'
acc_paris_int.loc[acc_paris_int['Num_Acc_x'].isin(acc_with_giveway['Num_Acc_x']), 'give_way'] = 'yes'''''''

In [33]:
acc_with_lights = gpd.sjoin(acc_lyon_buffered_gdf.drop_duplicates('Num_Acc_x'), lyon_traffic_signals.to_crs('EPSG:3035'))
# Create 'signalized' column and fill with 'yes' or 'no' based on the result of the spatial join
# Créer une colonne 'signalized' dans la GeoDataFrame intersections et assigner 'no' par défaut
acc_lyon_int['traffic_lights'] = 'no'

# Identifier les intersections avec des signaux et mettre à jour la colonne 'signalized'
acc_lyon_int.loc[acc_lyon_int['Num_Acc_x'].isin(acc_with_lights['Num_Acc_x']), 'traffic_lights'] = 'yes'

In [34]:
# Perform spatial join
acc_with_lights = gpd.sjoin(acc_buffered_gdf.drop_duplicates('Num_Acc_x'), paris_traffic_lights.to_crs('EPSG:3035'))
# Create 'signalized' column and fill with 'yes' or 'no' based on the result of the spatial join
# Créer une colonne 'signalized' dans la GeoDataFrame intersections et assigner 'no' par défaut
acc_paris_int['traffic_lights'] = 'no'

# Identifier les intersections avec des signaux et mettre à jour la colonne 'signalized'
acc_paris_int.loc[acc_paris_int['Num_Acc_x'].isin(acc_with_lights['Num_Acc_x']), 'traffic_lights'] = 'yes'

In [35]:
accidents_lyon=pd.merge(accidents_lyon,acc_lyon_int[['Num_Acc_x','stops','give_way','traffic_lights']].drop_duplicates('Num_Acc_x'),on='Num_Acc_x',how='left')
accidents_paris=pd.merge(accidents_paris,acc_paris_int[['Num_Acc_x','stops','give_way','traffic_lights']].drop_duplicates('Num_Acc_x'),on='Num_Acc_x',how='left')

## Tram tracks

In [36]:
tram=gpd.read_file('tram_crossing.geojson')
tram_crossing_paris=gpd.read_file('tram_crossing_paris.gpkg')


In [37]:
tram=tram.to_crs('EPSG:3035')
tram_crossing_paris=tram_crossing_paris.to_crs('EPSG:3035')

In [38]:
tram['geometry']=tram.buffer(25)

In [39]:
tram_crossing_paris['geometry']=tram_crossing_paris.buffer(25)

In [40]:
accidents_lyon.to_crs('EPSG:3035',inplace=True)

In [42]:
# Perform the spatial join
tram = tram[['railway', 'geometry']].reset_index(drop=True)

# Perform the spatial join
sjoin_result = gpd.sjoin(accidents_lyon.drop_duplicates('Num_Acc_x'), tram)

# Create a new column 'tram_crossing' and set it to 'yes' for those accidents_lyon

accidents_lyon['tram_crossing']='No'

accidents_lyon.loc[accidents_lyon['Num_Acc_x'].isin(sjoin_result['Num_Acc_x']), 'tram_crossing'] = 'yes'

In [43]:
accidents_paris.to_crs('EPSG:3035',inplace=True)

In [44]:
# Perform the spatial join
tram_crossing_paris = tram_crossing_paris[['railway', 'geometry']].reset_index(drop=True)


In [46]:
# Perform the spatial join
sjoin_result = gpd.sjoin(accidents_paris.drop_duplicates('Num_Acc_x'), tram_crossing_paris)


In [47]:

# Create a new column 'tram_crossing' and set it to 'yes' for those accidents_lyon

accidents_paris['tram_crossing']='No'

accidents_paris.loc[accidents_paris['Num_Acc_x'].isin(sjoin_result['Num_Acc_x']), 'tram_crossing'] = 'yes'

In [48]:
tram_tracks=gpd.read_file('tram_tracks_lyon.gpkg')
tram_tracks_paris=gpd.read_file('tram_tracks_paris.gpkg')

In [49]:
tram_tracks=tram_tracks.to_crs(accidents_lyon.crs)
tram_tracks['geometry']=tram_tracks.buffer(3)

In [50]:
tram_tracks_paris=tram_tracks_paris.to_crs(accidents_paris.crs)
tram_tracks_paris['geometry']=tram_tracks_paris.buffer(3)

In [51]:
accidents_lyon_with_tram=gpd.sjoin(accidents_lyon.drop_duplicates('Num_Acc_x'),tram_tracks[['railway','geometry']],how='left')

In [52]:
accidents_with_tram_paris=gpd.sjoin(accidents_paris.drop_duplicates('Num_Acc_x'),tram_tracks_paris[['railway','geometry']],how='left')

In [53]:
# Ajouter une colonne 'tram' avec des valeurs par défaut 'No'
accidents_lyon_with_tram['tram'] = 'No'

# Définir la valeur de 'tram' à 'Tram crossing' pour les lignes où 'tram_crossing' est égal à 'yes'
accidents_lyon_with_tram.loc[accidents_lyon_with_tram['tram_crossing'] == 'yes', 'tram'] = 'Tram crossing'

# Définir la valeur de 'tram' à 'On tracks' pour les lignes où 'railway' est égal à 'tram' et 'tram_crossing' est égal à 'No'
accidents_lyon_with_tram.loc[(accidents_lyon_with_tram['railway'] == 'tram') & (accidents_lyon_with_tram['tram_crossing'] == 'No'), 'tram'] = 'On tracks'


In [54]:
# Ajouter une colonne 'tram' avec des valeurs par défaut 'No'
accidents_with_tram_paris['tram'] = 'No'

# Définir la valeur de 'tram' à 'Tram crossing' pour les lignes où 'tram_crossing' est égal à 'yes'
accidents_with_tram_paris.loc[accidents_with_tram_paris['tram_crossing'] == 'yes', 'tram'] = 'Tram crossing'

# Définir la valeur de 'tram' à 'On tracks' pour les lignes où 'railway' est égal à 'tram' et 'tram_crossing' est égal à 'No'
accidents_with_tram_paris.loc[(accidents_with_tram_paris['railway'] == 'tram') & (accidents_with_tram_paris['tram_crossing'] == 'No'), 'tram'] = 'On tracks'

In [55]:
accidents_lyon=pd.merge(accidents_lyon,accidents_lyon_with_tram[['Num_Acc_x','tram']].drop_duplicates('Num_Acc_x'),on='Num_Acc_x',how='left')
accidents_paris=pd.merge(accidents_paris,accidents_with_tram_paris[['Num_Acc_x','tram']].drop_duplicates('Num_Acc_x'),on='Num_Acc_x',how='left')

In [56]:
accidents_lyon=accidents_lyon.drop(columns='adr')

In [57]:
accidents_lyon.to_crs('EPSG:4326').to_file('acci_lyon.geojson')

In [ ]:
accidents_paris.to_crs('EPSG:4326').to_file('acci_paris.geojson')
